# Internet archive 'big data' for 'small histories' 
* Outline of project
* reasoning 
* limitations 


In [9]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
wellcome_data = pd.read_table('wellcome-linkage.tsv', sep='|', names=['year', 'origin-domain', 'linked-domain'])


In [18]:
wellcome_data[:10]

,year,origin-domain,linked-domain
0,1996,odin.neuro.soton.ac.uk,www.wellcome.ac.uk\t1
1,1996,tarantula.wellcome.ac.uk,tarantula.wellcome.ac.uk\t24
2,1996,www.ability.org.uk,www.tmr.wellcome.ac.uk\t1
3,1996,www.healthindex.co.uk,www.tmr.wellcome.ac.uk\t1
4,1997,atuin.chime.ucl.ac.uk,www.wellcome.ac.uk\t10
5,1997,back.niss.ac.uk,www.wellcome.ac.uk\t1
6,1997,bushmills.mic.dundee.ac.uk,www.wellcome.ac.uk\t2
7,1997,com2.uclan.ac.uk,www.wellcome.ac.uk\t7
8,1997,epidemic.zoo.ox.ac.uk,www.tmr.wellcome.ac.uk\t2
9,1997,garbo.psychol.ucl.ac.uk,www.wellcome.ac.uk\t5
